In [1]:
import gc
import warnings
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold
warnings.filterwarnings("ignore")

In [2]:
#load data
data_path='./'
rating = pd.read_csv(data_path+'data/train/rating.txt',dtype=int,sep=' ',header=None,names=['uid','mid','rating'])
bigtag = pd.read_csv(data_path+'data/train/bigtag.txt',dtype=int,sep=' ',header=None,names=['uid','mid','tid'])
movie = pd.read_csv(data_path+'data/train/movie.txt',dtype=int,sep=' ',header=None,names=['mid','tag1','tag2','tag3','tag4','tag5','tag6','tag7','tag8'])
train = pd.read_csv(data_path+'data/valid/validation.txt',dtype=int,sep=' ',header=None,names=['uid','tid','y'])
test = pd.read_csv(data_path+'data/test.txt',dtype=int,sep=' ',header=None,names=['uid','tid'])

In [3]:
#FE
#将user和tag连接起来
def get_utid(df,uid='uid',tid='tid'):
    df['uid_tid'] = df[uid].astype(str)+"_"+df[tid].astype(str)
    
get_utid(train)
get_utid(test)
get_utid(bigtag)

In [6]:
#"merge rating&movie"
mov_rat = pd.merge(rating,movie,on='mid')
mov_rat_f = pd.concat([mov_rat[['uid','mid','rating',f'tag{i}']].rename(columns={f'tag{i}':'tid'}) for i in range(1,9)],ignore_index=True)
get_utid(mov_rat_f)

In [9]:
train['set'] = 'train'
test['set'] = 'test'
dff = pd.concat([train,test],ignore_index=True)

In [15]:
def fe_rating_gp_utid_stat(df):
    "rating&movie: rating groupby uid_tid stat"
    #total stat
    stat=['count','nunique','max','mean','median','min']
    res0 = mov_rat_f.groupby(['uid_tid'])['rating'].agg(stat)# 用户对标签的打分
    for s in stat:
        df[f'rating_gp_utid_{s}'] = df['uid_tid'].map(res0[s])
    #split tid stat
    for t in [f'tag{i}' for i in range(1,9)]:
        get_utid(mov_rat,uid='uid',tid=t)
        res0 = mov_rat.groupby(['uid_tid'])['rating'].agg(stat)
        for s in stat:
            df[f'rating_gp_u{t}_{s}'] = df['uid_tid'].map(res0[s])
        
fe_rating_gp_utid_stat(dff)

In [23]:
stat=['count','nunique','max','mean','median','min']
res0 = mov_rat_f.groupby(['uid_tid'])['rating'].agg(stat)# 用户对标签的打分

In [24]:
res0.head()

,count,nunique,max,mean,median,min
uid_tid,,,,,,
0_0,4,2,3,2.75,3.0,2
0_1047,1,1,1,1.00,1.0,1
0_1048,1,1,1,1.00,1.0,1
0_106,1,1,3,3.00,3.0,3
0_107,4,3,3,2.25,2.5,1


In [17]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uid                      6493 non-null   int32  
 1   tid                      6493 non-null   int32  
 2   y                        2039 non-null   float64
 3   uid_tid                  6493 non-null   object 
 4   set                      6493 non-null   object 
 5   rating_gp_utid_count     4362 non-null   float64
 6   rating_gp_utid_nunique   4362 non-null   float64
 7   rating_gp_utid_max       4362 non-null   float64
 8   rating_gp_utid_mean      4362 non-null   float64
 9   rating_gp_utid_median    4362 non-null   float64
 10  rating_gp_utid_min       4362 non-null   float64
 11  rating_gp_utag1_count    2397 non-null   float64
 12  rating_gp_utag1_nunique  2397 non-null   float64
 13  rating_gp_utag1_max      2397 non-null   float64
 14  rating_gp_utag1_mean    

In [19]:
#get mid-tid pairs
mov_tag = pd.concat([movie[['mid',f'tag{i}']].rename(columns={f'tag{i}':'tid'}) for i in range(1,9)],ignore_index=True)

In [20]:
def fe_tid_freq(df):
    "tag freq"
    #all movie tag freq 
    df['tid_freq'] = df['tid'].map(mov_tag.tid.value_counts())
    #split movie tag freq
    for c in ['tag1', 'tag2', 'tag3', 'tag4', 'tag5', 'tag6', 'tag7', 'tag8']:
        df[f'{c}_freq'] = df['tid'].map(movie[c].value_counts())

fe_tid_freq(dff)

In [21]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 68 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uid                      6493 non-null   int32  
 1   tid                      6493 non-null   int32  
 2   y                        2039 non-null   float64
 3   uid_tid                  6493 non-null   object 
 4   set                      6493 non-null   object 
 5   rating_gp_utid_count     4362 non-null   float64
 6   rating_gp_utid_nunique   4362 non-null   float64
 7   rating_gp_utid_max       4362 non-null   float64
 8   rating_gp_utid_mean      4362 non-null   float64
 9   rating_gp_utid_median    4362 non-null   float64
 10  rating_gp_utid_min       4362 non-null   float64
 11  rating_gp_utag1_count    2397 non-null   float64
 12  rating_gp_utag1_nunique  2397 non-null   float64
 13  rating_gp_utag1_max      2397 non-null   float64
 14  rating_gp_utag1_mean    

In [ ]:
def tag_fe(df,bigtag):
    "bigtag fe"
    bigtag_n1 = bigtag[bigtag.tid==-1]
    bigtag = bigtag[bigtag.tid>-1]
    get_utid(bigtag)
    
    # labeled tid stat
    stat = ['count','nunique']
    res0 = bigtag.groupby('tid')['uid'].agg(stat)# 每个tag有多少用户标记
    res1 = bigtag.groupby('tid')['mid'].agg(stat) # 每个tag有多少电影标记着
    res2 = bigtag.groupby('uid_tid')['mid'].agg(stat) #每对uid_tid对应多少电影
    for i in stat:
        df[f'bigtag_uid_gp_tid_{i}'] = df['tid'].map(res0[i])
        df[f'bigtag_mid_gp_tid_{i}'] = df['tid'].map(res1[i])
        df[f'bigtag_mid_gp_utid_{i}'] = df['uid_tid'].map(res2[i])    
    for c in ['uid_gp_tid','mid_gp_tid','mid_gp_utid']:
        df[f'bigtag_{c}_rate'] = df[f'bigtag_{c}_nunique']/df[f'bigtag_{c}_count'] #比率 
        df[f'bigtag_{c}_diff'] = df[f'bigtag_{c}_nunique']-df[f'bigtag_{c}_count']  #差值
        
    #graph uid-tid-mid-tid,like stat
    mov_tag1 = pd.merge(bigtag,mov_tag,on='mid')
    get_utid(mov_tag1,'uid','tid_y')
    like_uid_tid = mov_tag1.groupby('uid_tid')['mid'].agg(['count','nunique'])
    for s in like_uid_tid:
        df[f'graph_utmt_like_mid_gp_utid_{s}'] = df['uid_tid'].map(like_uid_tid[s])
    df['graph_utmt_like_mid_gp_utid_diff'] = df['graph_utmt_like_mid_gp_utid_nunique']-df['graph_utmt_like_mid_gp_utid_count']
    df['graph_utmt_like_mid_gp_utid_rate'] = df['graph_utmt_like_mid_gp_utid_nunique']/df['graph_utmt_like_mid_gp_utid_count']
    df['graph_utmt_like_tid_freq'] = df['tid'].map(mov_tag1.tid_y.value_counts(normalize=True)).fillna(-0.01)

    # dislike stat
    mov_tag2 = pd.merge(bigtag_n1,mov_tag,on='mid')
    get_utid(mov_tag2,'uid','tid_y')
    hate_uid_tid = mov_tag2.groupby('uid_tid')['mid'].agg(['count','nunique'])
    for s in hate_uid_tid:
        df[f'graph_utmt_dislike_mid_gp_utid_{s}'] = df['uid_tid'].map(hate_uid_tid[s])
    df['graph_utmt_dislike_mid_gp_utid_diff'] = df['graph_utmt_dislike_mid_gp_utid_nunique']-df['graph_utmt_dislike_mid_gp_utid_count']
    df['graph_utmt_dislike_mid_gp_utid_rate'] = df['graph_utmt_dislike_mid_gp_utid_nunique']/df['graph_utmt_dislike_mid_gp_utid_count']
    df['graph_utmt_dislike_tid_freq'] = df['tid'].map(mov_tag2.tid_y.value_counts(normalize=True)).fillna(-0.01)

    # graph uid-tid-uid-tid
    utut = pd.merge(bigtag[['uid','tid']],bigtag[['uid','tid']],on='tid')
    utut = utut[utut.uid_x!=utut.uid_y]
    del utut['tid'];gc.collect()
    utut = pd.merge(utut,bigtag[['uid','tid']],left_on='uid_y',right_on='uid')
    del utut['uid_y'];gc.collect()
    utut['uid_tid'] = utut['uid_x'].astype(str)+"_"+utut['tid'].astype(str)
    res = utut.groupby('uid_tid')['uid'].agg(['count','nunique'])
    for s in res:
        df[f'graph_utut_{s}'] = df['uid_tid'].map(res[s])
    df['graph_utut_nunique_rate'] = df['graph_utut_nunique']/df['graph_utut_count']
    df['graph_utut_nunique_diff'] = df['graph_utut_nunique']-df['graph_utut_count']
    df['graph_utut_freq']= df['tid'].map(utut.tid.value_counts(normalize=True)).fillna(-0.01)
    del utut;gc.collect()

In [ ]:
tag_fe(dff,bigtag)
